# SQL query from table names - Continued

In [13]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [14]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [15]:
context = [
    {'role': 'system', 'content': """
    You are an expert game developer and database designer.
    Your task is to create at least 3 SQL tables related to video games.
    The tables should cover different aspects of a game system, such as:
    - Games table (with game id, title, genre, release year)
    - Players table (with player id, name, country, and rank)
    - Scores table (with score id, player id, game id, score value, and date)
    You can add more tables if needed (like Achievements, Reviews, or Tournaments).
    Provide clean and well-structured SQL CREATE TABLE statements.
    """}
]


In [ ]:
context.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following types of questions based on the tables provided:

 Get all games released after 2018
SELECT * FROM Games WHERE release_year > 2018;

 Find all players from 'USA'
SELECT * FROM Players WHERE country = 'USA';

 Get the highest score for each game
SELECT game_id, MAX(score_value) as highest_score 
FROM Scores 
GROUP BY game_id;

 List the top 3 players by total score
SELECT player_id, SUM(score_value) as total_score 
FROM Scores 
GROUP BY player_id 
ORDER BY total_score DESC 
LIMIT 3;

 Count how many players are from each country
SELECT country, COUNT(*) as number_of_players 
FROM Players 
GROUP BY country;
"""})


In [17]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [18]:
#new
context_user = context.copy()
print(return_CCRMSQL("""List all games released after 2020""", context_user))

```sql
SELECT * FROM Games WHERE release_year > 2020;
```


In [19]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Write a review of the most profitable games", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM games
WHERE profit = (SELECT MAX(profit) FROM games);
```

This SQL query selects all the games from the "games" table that have the maximum profit value, showing the most profitable game(s).


In [20]:
#new
print(return_CCRMSQL("Write a review of the worst games?", context_user))

As an AI assistant, I don't have personal opinions or the ability to write reviews. However, I can provide you with a hypothetical scenario where reviews of the worst games could be stored in a database table. 

Let's create a table called `GameReviews` to store reviews of games. Each review can have details like the game title, reviewer name, review text, rating, and date of the review. Here is the SQL CREATE TABLE statement for the `GameReviews` table:

```sql
CREATE TABLE GameReviews (
    review_id INTEGER PRIMARY KEY,
    game_id INTEGER,
    reviewer_name TEXT,
    review_text TEXT,
    rating INTEGER,
    review_date DATE,
    FOREIGN KEY (game_id) REFERENCES Games(game_id)
);
```

In this table, you can store reviews of games, including the worst games, by inserting relevant data into the table. You can then query this table to retrieve reviews of the worst games based on low ratings or negative comments.

If you have specific games or reviews in mind, feel free to provide more

In [21]:
#old
print(return_CCRMSQL("Get the top 5 players by their total scores", old_context_user))

This is your SQL:
```sql
SELECT ID_usr, SUM(score) AS total_score
FROM players
GROUP BY ID_usr
ORDER BY total_score DESC
LIMIT 5;
```

This SQL query selects the player IDs and their total scores from the "players" table, calculates the sum of scores for each player, groups the results by player ID, orders them in descending order based on the total score, and limits the output to the top 5 players.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

# 1

In [22]:
context1 = [
    {'role': 'system', 'content': """
    You are an expert restaurant management system designer and database architect.
    Your task is to create at least 3 SQL tables related to a restaurant system.
    The tables should cover different aspects of a restaurant system, such as:
    - Menu table (with menu_id, item_name, category, price)
    - Customers table (with customer_id, name, phone, and email)
    - Orders table (with order_id, customer_id, menu_id, quantity, and order_date)
    You can add more tables if needed (like Reservations, Reviews, or Payments).
    Provide clean and well-structured SQL CREATE TABLE statements.
    """}
]


In [23]:
context1.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following types of questions based on the tables provided:

Get all menu items that cost more than 50
SELECT * FROM Menu WHERE price > 50;

Find all customers from 'Riyadh'
SELECT * FROM Customers WHERE city = 'Riyadh';

Get the highest order quantity for each menu item
SELECT menu_id, MAX(quantity) as highest_quantity 
FROM Orders 
GROUP BY menu_id;

List the top 3 customers by total amount spent
SELECT customer_id, SUM(quantity * price) as total_spent 
FROM Orders 
JOIN Menu ON Orders.menu_id = Menu.menu_id 
GROUP BY customer_id 
ORDER BY total_spent DESC 
LIMIT 3;

Count how many customers are from each city
SELECT city, COUNT(*) as number_of_customers 
FROM Customers 
GROUP BY city;
"""})


In [24]:
#Functio to call the model.
def return_CCRMSQL(user_message, context1):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context1.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [25]:
#new
context_user = context1.copy()
print(return_CCRMSQL("""List all dishes with price greater than 50""", context_user))

```sql
SELECT * FROM Menu WHERE price > 50;
```


In [26]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List all dishes with price greater than 50", old_context_user))

This is your SQL:
```sql
SELECT * FROM dishes WHERE price > 50;
```

This SQL command selects all the dishes from the table "dishes" where the price is greater than 50.


# 2

In [27]:
context2 = [
    {'role': 'system', 'content': """
    You are an expert bookstore management system designer and database architect.
    Your task is to create at least 3 SQL tables related to a bookstore system.
    The tables should cover different aspects of a bookstore system, such as:
    - Books table (with book_id, title, author, genre, price, and publication_year)
    - Customers table (with customer_id, name, phone, and email)
    - Orders table (with order_id, customer_id, book_id, quantity, and order_date)
    You can add more tables if needed (like Reviews, Publishers, or Payments).
    Provide clean and well-structured SQL CREATE TABLE statements.
    """}
]


In [28]:
context2.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following types of questions based on the tables provided:

Get all books that cost more than 50
SELECT * FROM Books WHERE price > 50;

Find all customers from 'Riyadh'
SELECT * FROM Customers WHERE city = 'Riyadh';

Get the highest quantity ordered for each book
SELECT book_id, MAX(quantity) as highest_quantity 
FROM Orders 
GROUP BY book_id;

List the top 3 customers by total amount spent
SELECT customer_id, SUM(quantity * price) as total_spent 
FROM Orders 
JOIN Books ON Orders.book_id = Books.book_id 
GROUP BY customer_id 
ORDER BY total_spent DESC 
LIMIT 3;

Count how many customers are from each city
SELECT city, COUNT(*) as number_of_customers 
FROM Customers 
GROUP BY city;
"""})


In [29]:
#Functio to call the model.
def return_CCRMSQL(user_message, context2):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context2.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [30]:
#new
context_user = context2.copy()
print(return_CCRMSQL("""Count how many customers are from each city""", context_user))

```sql
SELECT city, COUNT(*) as number_of_customers 
FROM Customers 
GROUP BY city;
```


In [31]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Count how many customers are from each city", old_context_user))

This is your SQL:
```sql
SELECT city, COUNT(*) AS customer_count
FROM customers
GROUP BY city;
```

This SQL query selects the city column from the customers table and counts the number of customers in each city by using the COUNT(*) function. The result is grouped by city using the GROUP BY clause.


# 3

In [32]:
context3 = [
    {'role': 'system', 'content': """
    You are an expert supermarket management system designer and database architect.
    Your task is to create at least 3 SQL tables related to a supermarket system.
    The tables should cover different aspects of a supermarket system, such as:
    - Products table (with product_id, name, category, price, and expiration_date)
    - Customers table (with customer_id, name, phone, and email)
    - Orders table (with order_id, customer_id, product_id, quantity, and order_date)
    You can add more tables if needed (like Reviews, Suppliers, or Payments).
    Provide clean and well-structured SQL CREATE TABLE statements.
    """}
]


In [33]:
context3.append({'role': 'system', 'content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following types of questions based on the tables provided:

Get all products that cost more than 50
SELECT * FROM Products WHERE price > 50;

Find all customers from 'Riyadh'
SELECT * FROM Customers WHERE city = 'Riyadh';

Get the highest quantity ordered for each product
SELECT product_id, MAX(quantity) as highest_quantity 
FROM Orders 
GROUP BY product_id;

List the top 3 customers by total amount spent
SELECT customer_id, SUM(quantity * price) as total_spent 
FROM Orders 
JOIN Products ON Orders.product_id = Products.product_id 
GROUP BY customer_id 
ORDER BY total_spent DESC 
LIMIT 3;

Count how many customers are from each city
SELECT city, COUNT(*) as number_of_customers 
FROM Customers 
GROUP BY city;
"""})


In [34]:
#Functio to call the model.
def return_CCRMSQL(user_message, context3):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context3.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [ ]:
#new
context_user = context3.copy()
print(return_CCRMSQL("""List all dishes with price less than 50""", context_user))

SELECT * FROM Products WHERE price < 50;


In [ ]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List all dishes with price less than 50", old_context_user))

This is your SQL:
```sql
SELECT * FROM dishes WHERE price < 50;
```
This SQL query selects all dishes from the table "dishes" where the price is less than 50.


# Report
# Objective
We designed and tested multiple SQL database system contexts for different domains:

Video Games

Restaurant

Bookstore

Supermarket



# Each context included:

SQL CREATE TABLE definitions

Example SQL queries for retrieving meaningful data

Context-specific logic for business operations

Then, we tested GPT’s ability to generate valid and relevant SQL queries based on these contexts.

# Findings & Observations
# 1. Video Games Context
GPT successfully generated clean tables like Games, Players, and Scores. The queries for top players and highest scores were accurate and logically correct.

 

# 2. Restaurant Context
The system produced appropriate tables (Menu, Customers, Orders) and accurate queries for the most expensive menu items, customers by city, and top spenders

# 3. Bookstore Context
The queries for top-spending customers, highest ordered books, and high-priced books made sense and were valid.




# 4. Supermarket Context
The queries for top-spending customers, highest ordered products, and high-priced products were all relevant and valid. They captured the right data, reflecting the business needs.


# Lessons Learned
Prompt Clarity Matters:
Precise, well-structured, domain-specific prompts result in more reliable and accurate SQL generation.

Risk of Hallucination in Text Outputs:
While SQL generation was reliable, free-text generations mixed facts and fiction.



Always Validate SQL:
It’s important to review both the table definitions and generated queries to make sure they match in terms of field names and logic.

# Conclusion
This experiment proved that GPT can reliably generate structured SQL schemas and queries when guided with clear and specific instructions. The main issues appeared in:

Free-text generations (hallucinations)

Schema migration errors when switching domains

For future improvements:

Use schema validators

Enforce stricter schema-query consistency

Add factual verification for free-text outputs